<a href="https://colab.research.google.com/github/Vaibhavgoel123/Artificial-Intelligence/blob/Tensor/Pytorch_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

Mounting the Google Drive in the environment.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing untitled2.py to process the outputs.

In [ ]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/')

import untitled2
import importlib
importlib.reload(untitled2)

print("Successfully imported and reloaded untitled2!")

Successfully imported and reloaded untitled2!


In [ ]:
torch.manual_seed(42)

# Torch Tensors


Tensor Shapes

In [3]:
#Delivery Data
distances = torch.tensor([[3.0],[7.0],[12.0],[18.0],[22.0],[28.0]])
print(distances.shape)
simple_model=nn.Linear(1,1)
output=simple_model(distances)

torch.Size([6, 1])


This runs because PyTorch Linear model consider the first 1 as the dimension so fist 1 is dimension and the other 1 is the number of features.

Tensor Data Types



1.   float64 -> Extra Precision
2.   int8 -> Memory Efficiency


1.   float32 -> Default and works reliably
2.   int64 -> Mostly used





In [4]:
int_tensor = torch.tensor([1,2,3])
print(int_tensor.dtype)

torch.int64


In [5]:
float_tensor = torch.tensor([1.0,2.0,3.0])
print(float_tensor.dtype)

torch.float32


In [6]:
float_tensor = torch.tensor([1,2,3],dtype=torch.float32)
print(float_tensor)

tensor([1., 2., 3.])


In [7]:
float_tensor = int_tensor.float()

In [8]:
mixed_tensor = float_tensor + int_tensor
print(f"mixed type: {mixed_tensor.dtype}")

mixed type: torch.float32


In [9]:
import numpy as np
numpy_array = np.array([1,2,3])
numpy_to_tensor = torch.from_numpy(numpy_array)
print(f"numpy_array type:{numpy_array.dtype}, tensor type:{numpy_to_tensor.dtype}")

numpy_array type:int64, tensor type:torch.int64


To create torch matrix

In [10]:
zero_matrix = torch.zeros(3,3,dtype=torch.int64)
ones_matrix = torch.ones(2,4)
random_matrix = torch.rand(5,5,dtype=torch.float32)

In [11]:
print(zero_matrix)
print(ones_matrix)
print(random_matrix)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0.2307, 0.5390, 0.9426, 0.6466, 0.4201],
        [0.3421, 0.4181, 0.4401, 0.1464, 0.0154],
        [0.3073, 0.5891, 0.4742, 0.3943, 0.8912],
        [0.7299, 0.9898, 0.1169, 0.7808, 0.1834],
        [0.1335, 0.7347, 0.5440, 0.7241, 0.6778]])


Reshaping Tensors

In [12]:
#Sample Data
single_distance = torch.tensor(25.0)
#Adding one dimension
with_dist = single_distance.unsqueeze(0)
#Model ready dim
ready = with_dist.unsqueeze(1)

ready.shape

squeezed = ready.squeeze()
squeezed.shape
squeezed

tensor(25.)

Indexing and Slicing

In [13]:
predictions = torch.tensor([[14.9],[24.1],[34.6],[45.2]])
value = predictions[0].item()
print(value)

14.899999618530273


#Tensor Maths

Calculating weights involves multiple operations and matrix multiplication or scalar multiplication is one of them.

Multiplying small matrix is fine but when talking about large matrix to multiply single array to large matrix we need to create a matrix of that same size of the elements of that array so it increases the complexity.

To overcome this **Broadcasting** method was introduced.
Broadcasting means to repeat the values to match the dimensions of the corresponding matrix.

**PyTorch** automatically expands the dimensions in the case of scaler operations so in broadcasting the Pytorch compares the dimensions of both entities and try to match them, if anyone entity matches perfectly then it try to match the other one by repeating the values.


In [17]:
distances = [[3.0],[7.0],[12.0]]
weight = 2.3
bias = 8.0
